In [2]:
import pandas as pd
import sqlite3
import re
from contextlib import closing
from datetime import datetime

In [28]:
db = "test 2.db"

def get_uid(name:str)->str:
    return re.findall(r"\d+", name)

def get_date(name:int)->datetime:
    return datetime.strptime(
        re.findall(r"\d+", str(name))[0],
        r'%Y%m%d%H%M%S'
    ).strftime(r"%c")

def list_sessions():
    with closing(sqlite3.connect(db)) as conn:
        sessions = pd.read_sql(
            "SELECT id, notes FROM session WHERE (notes is not NULL) and (notes is not \"\") and (sensors == 5)", conn
            )
        sessions["date"] = sessions["id"].apply(get_date)

        return sessions

def get_session(uid:int):
    with closing(sqlite3.connect(db)) as conn:
        return pd.read_sql(
            f"SELECT * FROM session_{uid}  WHERE (signal_strength0 == 0) and (signal_strength1 == 0) and (signal_strength2 == 0) and (signal_strength3 == 0) and (signal_strength4 == 0)", conn
        )

In [24]:
sessions = list_sessions()
sessions

,id,notes,date
0,20240824222007,unu,Sat Aug 24 22:20:07 2024
1,20240824223239,pruebas,Sat Aug 24 22:32:39 2024
2,20240824223638,nada también,Sat Aug 24 22:36:38 2024
3,20240824223904,prueba,Sat Aug 24 22:39:04 2024
4,20240824225005,sexo,Sat Aug 24 22:50:05 2024
...,...,...,...
79,20241107145351,sofi 1,Thu Nov 7 14:53:51 2024
80,20241107150554,sofi 3,Thu Nov 7 15:05:54 2024
81,20241107151958,miranda5,Thu Nov 7 15:19:58 2024
82,20241130090030,Marcelo,Sat Nov 30 09:00:30 2024


In [19]:
session = get_session(20240824223638)
session.loc[session['signal_strength0'] == 0]

,signal_strength0,attention0,meditation0,delta0,theta0,low_alpha0,high_alpha0,low_beta0,high_beta0,low_gamma0,...,attention1,meditation1,delta1,theta1,low_alpha1,high_alpha1,low_beta1,high_beta1,low_gamma1,high_gamma1


In [31]:
sessions = list_sessions()
size = sessions.index.size
for i in range(size):
    with closing(sqlite3.connect(db)) as conn:
        session = get_session(sessions.iloc[i,0])
        session.to_sql(name='datos', con=conn , if_exists='append')